# Код для извлечения чистого текста из html-кода с сохранением его структуры (абзацы, заголовки).

Этот код позволяет извлекать текст из html кода с пометками соответствующих абзацев. Список абзацев можно найти в массивах headings и paragraphs. Под каждый из массивов есть условный знак, с которым хранится абзац соответствующего типа. При желании его можно разнообразить.

In [56]:
import os
import warnings

from bs4 import BeautifulSoup

import requests
import urllib3
from urllib.request import urlopen
from urllib.parse import urlparse

import pandas as pd
import numpy as np

import sqlite3


warnings.filterwarnings("ignore")
urllib3.disable_warnings()

In [57]:
sample_html_code = '''
<html>
  <body>
    <h1>This is a heading</h1>
    <p>This is a paragraph.</p>
    <p>Another paragraph.</p>
  </body>
</html>
'''

def MarkText(html_code : str):

    soup = BeautifulSoup(html_code, 'html.parser')

    headings = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
    paragraphs = ['p', 'pre']

    HEADING_MARK = '//@' #  При желании, можно поменять.
    PARAGRAPH_MARK = '//&'

    def process_tag(tag):
        if tag.name in headings:
            return HEADING_MARK + tag.text.strip() + '\n'
        elif tag.name in paragraphs:
            return PARAGRAPH_MARK + tag.text.strip() + '\n'
        else:
            return ''


    result = ''

    for tag in soup.find_all():
        result += process_tag(tag)

    return result
print(MarkText(sample_html_code))

//@This is a heading
//&This is a paragraph.
//&Another paragraph.



# Код для парсинга веб сайтов с сохранением иерархической связи между страницами.

Здесь мы подключаемся к удалённой реляционной базе данных, создаём базу pages с полями, соответствующими логике ниже и создаём курсор для итерации по базе данных.

In [58]:
conn = sqlite3.connect('history.db')
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS Pages (
    PageID INT PRIMARY KEY,
    ParentPageID INT,
    PageLevel INT,
    PageURL TEXT,
    PageText TEXT,
    CompanyName TEXT,
    RatingRAEX TEXT
    )
''')


Загрузим таблицу из агенства RAEX, в которой есть необходимые нам ссылки.

In [59]:
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
    "accept": "application/json"
}

timeout = (5, 25)
RAEX_PATH = '/content/RAEX list.xlsx'
raex_list = pd.read_excel(RAEX_PATH).dropna(subset=["url_sustainability"]).reset_index(drop=True)
raex_list

,№,Название,Код MOEX,Подотрасль,ESG-рейтинг,E Rank,E-рейтинг,S Rank,S-рейтинг,G Rank,G-рейтинг,Год последней оцененной отчетности,url,url_sustainability
0,1,НЛМК,NLMK,Чёрная металлургия,AA,2,AA,2,AA,21,A,2021,https://nlmk.com/ru/,https://nlmk.com/ru/sustainability/
1,2,«Полюс»,PLZL,Драгоценные металлы,AA,1,AAA,14,A,27,A,2021,https://polyus.com/ru/,https://sustainability.polyus.com/ru/
2,3,«Уралкалий»,-,Агрохимикаты,A,6,BBB,1,AA,6,AA,2021,https://www.uralkali.com/ru/,https://www.uralkali.com/ru/sustainability/
3,4,«ЭЛ5-Энерго»,ELFV,Электроэнергетика,A,9,BBB,9,A,2,AAA,2021,https://www.el5-energo.ru,https://www.el5-energo.ru/sustainability/
4,5,«Полиметалл»,POLY,Драгоценные металлы,A,4,A,5,A,12,AA,2021,https://www.polymetalinternational.com/ru/,https://www.polymetalinternational.com/ru/sust...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,153,Кордиант,-,Производство шин,C,154,C,154,C,142,CC,2020,https://cordiant.ru,https://www.cordiant-tyre.ru/values/
137,155,«Промышленно-металлургический холдинг» (ПМХ),-,Чёрная металлургия,C,156,C,148,CC,149,CC,2020,https://www.metholding.ru,https://www.metholding.ru/development/
138,156,"""Титан"", группа компаний (деревообработка)",-,Деревообработка,C,148,C,152,C,152-154,CC,2020,https://titan-group.ru,https://titan-group.ru/about/development/
139,157,«Минудобрения»,-,Агрохимикаты,C,157,C,157,C,157-160,C,2021,https://minudo.ru,https://minudo.ru/?cid=28&parent_id=5


Напишем функцию для парсинга сайта. В аргументах зададим компанию (как набор признаков), ссылку на сайт и максимальную глубину обхода. Будем обходить все ссылки на сайте обходом в ширину, и считывать (на этом этапе) весь текст сайта, маркируя его заголовки и абзацы специальными символами.

Зададим константу CROP_TO_SHOW = 50, чтобы не сохранять всю страницу и показывать только первые 50 считанных символов (при желании, можно менять).

In [60]:
CROP_TO_SHOW = 50

from urllib.parse import urljoin

def parsing_site_demo(company, url, max_depth=0):
    page_id = cur.lastrowid if cur.lastrowid != None else 0

    internalLinks = [(url, '/', 0, -1)] #  Создаем массив ссылок, которые обрабатываем обходом в ширину.
    used = []
    path_from = {}
    megadata = pd.DataFrame()

    while internalLinks:
        data_to_csv = ()

        now = internalLinks.pop(0)
        url, from_url, depth, id_from = now
        # url = now[0]
        # from_url = now[1]
        # depth = now[2]
        # id_from = now[3]

        #  Помечаем ссылку как использованную, чтоб не зациклиться.
        used.append(url)
        # Сохраняем ссылку откуда пришли, чтобы потом восстановить путь до корня.
        path_from[url] = from_url

        response = requests.get(url=url, timeout=timeout, headers=headers, verify=False)
        text = MarkText(response.text)[:CROP_TO_SHOW]
        data = pd.DataFrame()
        data['company'] = [company['Название']]
        data['url'] = [url]
        data['text'] = [text]

        soup = BeautifulSoup(response.content, 'lxml')

        # data = data[['rating', 'company', 'url', 'text']]
        # По идее, в конце работы программы, в megadata должно храниться ваще все.

        # Считываем все ссылки на сайте и добавляем их в конец нашей очереди обхода в ширину.
        # Проверяем так же глубину обхода - слишком далеко нам уходить не надо.
        if depth < max_depth:
            NewLinks = [
                a.get('href') for a in soup.find_all('a')
                if a.get('href') and a.get('href').startswith('/')
            ]
            print(NewLinks)
            NewLinks = [link[1:] for link in NewLinks]
            NewLinks = [*set(NewLinks)]
            NewLinks = list(filter(None, NewLinks))

            for link in NewLinks:
                    if ".pdf" not in link and ".PDF" not in link:

                        new_url = urljoin(url[:url.rfind('.com')] + '.com', link)
                        print(new_url)
                        #Тут как раз добавляем URL в конец очереди.
                        if new_url not in used:
                            internalLinks.append((new_url, url, depth + 1, page_id))


        data_to_csv = (page_id, id_from, depth, url, text, company['Название'])
        #  Путь до корневой папки и будет иерархической определяющей нашей страницы.
        #  Восстанавливается он следующим образом :
        path_now = []
        print(url, 'downloaded successfully!')
        while url != '/' :
            path_now.append(url)
            url = path_from[url]
        path_now.reverse()
        data['path'] = path_now
        data['depth'] = depth
        #  В path_now хранится путь до страницы из корня!

        cur.execute("INSERT OR IGNORE INTO Pages (PageID, ParentPageID, PageLevel, PageURL,"
                    " PageText, CompanyName) VALUES (?, ?, ?, ?, ?, ?)", data_to_csv)

        megadata = pd.concat([megadata, data])

        #  Добавляем извлеченные данные как в sql таблицу, так и в датафрейм.
        #  Кто знает, что нам может пригодиться...)

        page_id += 1
    return megadata

Протестируем наш код на 10 первых сайтах из таблицы RAEX.

(Можно менять в константе SITES_TO_PARSE)

Здесь так же можно настроить переменную max_depth, которая отвечает за глубину обхода страниц. Но даже при значении max_depth = 1, код будет работать довольно долго.



In [61]:
max_depth = 0
SITES_TO_PARSE = 10

big_df = pd.DataFrame()
for i, company in raex_list.iterrows():
    rating = company['№']
    name = company['Название']
    url = company['url_sustainability']
    df = parsing_site_demo(company, url, max_depth)
    big_df = pd.concat([big_df, df])
    if (i == SITES_TO_PARSE - 1) :
        print('Демо-версия окончена! Дальше надо платить деньги. :D')
        break

conn.commit()

big_df


https://nlmk.com/ru/sustainability/ downloaded successfully!
https://sustainability.polyus.com/ru/ downloaded successfully!
https://www.uralkali.com/ru/sustainability/ downloaded successfully!
https://www.el5-energo.ru/sustainability/ downloaded successfully!
https://www.polymetalinternational.com/ru/sustainability/ downloaded successfully!
https://www.phosagro.ru/sustainability/ downloaded successfully!
https://ir.mkb.ru/sustainability downloaded successfully!
https://www.nornickel.ru/sustainability/ downloaded successfully!
https://severstal.com/rus/sustainable-development/ downloaded successfully!
https://lukoil.ru/Sustainability downloaded successfully!
Демо-версия окончена! Дальше надо платить деньги. :D


,company,url,text,path,depth
0,НЛМК,https://nlmk.com/ru/sustainability/,//@Устойчивое развитие\n//@Наш подход\n//&Явля...,https://nlmk.com/ru/sustainability/,0
0,«Полюс»,https://sustainability.polyus.com/ru/,//&RAEX подтвердило ESG-рейтинг «Полюса» на ур...,https://sustainability.polyus.com/ru/,0
0,«Уралкалий»,https://www.uralkali.com/ru/sustainability/,,https://www.uralkali.com/ru/sustainability/,0
0,«ЭЛ5-Энерго»,https://www.el5-energo.ru/sustainability/,//@Активы\n//@Руководство\n//@Карьера\n//@Гене...,https://www.el5-energo.ru/sustainability/,0
0,«Полиметалл»,https://www.polymetalinternational.com/ru/sust...,//@Устойчивое развитие\n//&В свете произошедши...,https://www.polymetalinternational.com/ru/sust...,0
0,"«ФосАгро», группа",https://www.phosagro.ru/sustainability/,//@Устойчивое развитие\n//@Управление устойчив...,https://www.phosagro.ru/sustainability/,0
0,«МОСКОВСКИЙ КРЕДИТНЫЙ БАНК»,https://ir.mkb.ru/sustainability,//@С заботой о будущем\n,https://ir.mkb.ru/sustainability,0
0,"«Норильский никель», горно-металлургическая ко...",https://www.nornickel.ru/sustainability/,,https://www.nornickel.ru/sustainability/,0
0,«Северсталь»,https://severstal.com/rus/sustainable-developm...,//@Устойчивое\r\nразвитие\n//@Отчет об устойчи...,https://severstal.com/rus/sustainable-developm...,0
0,«ЛУКОЙЛ»,https://lukoil.ru/Sustainability,//@block\n//@Web Page Blocked!\n//&The page ca...,https://lukoil.ru/Sustainability,0


Посмотрим, что лежит в нашей SQL таблице?

In [62]:
cur.execute("SELECT * FROM Pages")
rows = cur.fetchall()

for row in rows:
    print(row)

conn.close()

(0, -1, 0, 'https://nlmk.com/ru/sustainability/', '//@Устойчивое развитие\n//@Наш подход\n//&Являясь од', 'НЛМК', None)
(1, -1, 0, 'https://sustainability.polyus.com/ru/', '//&RAEX подтвердило ESG-рейтинг «Полюса» на уровне', '«Полюс»', None)
(2, -1, 0, 'https://www.uralkali.com/ru/sustainability/', '', '«Уралкалий»', None)
(3, -1, 0, 'https://www.el5-energo.ru/sustainability/', '//@Активы\n//@Руководство\n//@Карьера\n//@Генерация э', '«ЭЛ5-Энерго»', None)
(4, -1, 0, 'https://www.polymetalinternational.com/ru/sustainability/', '//@Устойчивое развитие\n//&В свете произошедших соб', '«Полиметалл»', None)
(5, -1, 0, 'https://www.phosagro.ru/sustainability/', '//@Устойчивое развитие\n//@Управление устойчивым ра', '«ФосАгро», группа', None)
(6, -1, 0, 'https://ir.mkb.ru/sustainability', '//@С заботой о\xa0будущем\n', '«МОСКОВСКИЙ КРЕДИТНЫЙ БАНК»', None)
(7, -1, 0, 'https://www.nornickel.ru/sustainability/', '', '«Норильский никель», горно-металлургическая компания', None)
(8, -1, 0, 'https: